In [1]:
from webscrapping.database.sql_queries import ValorantQueries
import pandas as pd

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [3]:
states = vq.get_initial_state()
events = vq.get_event_table()

# Json Style

In [11]:
chosen_round = 5
round_states = states[(states["Round"] == chosen_round)]
round_states = round_states.sort_values(by=["Player Side"])
round_events = events[(events["Round"] == chosen_round)]
alive_info = {item: "alive" for item in round_states["Player Name"].unique()}
player_info = {item: {} for item in round_states["Player Name"].unique()}
round_winner_table = vq.round_winner_dataframe()
round_winner = int(round_winner_table[round_winner_table["round_number"] == chosen_round]["winning_team"])
round_winner
# Get round_winner_table where round_number == chosen_round

53

In [22]:
event_amount = round_events.shape[0]
alive_dict = {r: {} for r in range(1, event_amount+1)}
alive_event_info = {item: "alive" for item in round_states["Player Name"].unique()}
for row_index in range(len(round_events)):
    row = round_events.iloc[row_index]
    event_index = row["EventIndex"]
    victim = row["VictimName"]
    event_type = row["EventType"]
    if event_type == "kill":
        alive_event_info[victim] = "dead"
    elif event_type == "revive":
        alive_event_info[victim] = "alive"
    alive_dict[event_index] = alive_event_info
round_states

,Match ID,Round ID,Round,Player ID,Player Name,Status,Team ID,Player Side,Remaining Creds,Loadout Value,...,Weapon Name,Weapon Price,Utility Value,Agent Name,Starting Side,Controller,Duelist,Initiator,Sentinel,Has Operator
43,43621,695328,5,1877,frz,alive,53,attack,1450,4800,...,Vandal,2900,4800,Reyna,attack,0,1,0,0,False
44,43621,695328,5,3105,gtnziN,alive,53,attack,4250,4600,...,Vandal,2900,4600,Jett,attack,0,1,0,0,False
47,43621,695328,5,2895,Saadhak,alive,53,attack,2450,5500,...,Phantom,2900,5500,Sage,attack,0,0,0,1,False
48,43621,695328,5,2294,Sacy,alive,53,attack,4600,4600,...,Phantom,2900,4600,Sova,attack,0,0,1,0,False
49,43621,695328,5,194,sutecas,alive,53,attack,3150,4500,...,Vandal,2900,4500,Viper,attack,1,0,0,0,False
40,43621,695328,5,1843,ade,alive,444,defense,200,3700,...,Phantom,2900,3700,Viper,defense,1,0,0,0,False
41,43621,695328,5,1075,Bazzi,alive,444,defense,0,4600,...,Vandal,2900,4600,Sova,defense,0,0,1,0,False
42,43621,695328,5,1824,Fisker,alive,444,defense,200,5650,...,Operator,5000,5650,Jett,defense,0,1,0,0,True
45,43621,695328,5,876,Munchkin,alive,444,defense,250,4600,...,Vandal,2900,4600,KAY/O,defense,0,0,1,0,False
46,43621,695328,5,2388,neth,alive,444,defense,100,4700,...,Phantom,2900,4700,Sage,defense,0,0,0,1,False


In [25]:
round_states.iloc[0]

Match ID            43621
Round ID           695328
Round                   5
Player ID            1877
Player Name           frz
Status              alive
Team ID                53
Player Side        attack
Remaining Creds      1450
Loadout Value        4800
Agent ID               10
Weapon ID               4
Armor ID                2
Shield                 50
Shield Price         1000
Weapon Name        Vandal
Weapon Price         2900
Utility Value        4800
Agent Name          Reyna
Starting Side      attack
Controller              0
Duelist                 1
Initiator               0
Sentinel                0
Has Operator        False
Name: 43, dtype: object